**BERT**

In [3]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install torch
!pip install tensorflow

In [5]:
import json
import torch
from transformers import BertTokenizer, BertModel

# Load the pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Load the JSON file containing questions and answers
with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)

# User's question
user_question = input("Ask a question: ")

# Tokenize and encode the user's question
user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

# Calculate cosine similarity between the user's question and all questions in the JSON file
similarities = []
for item in data:
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)

    # Calculate cosine similarity
    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding)
    similarities.append(cosine_sim.item())

# Find the index of the question with the highest cosine similarity
max_index = similarities.index(max(similarities))

# Get the answer corresponding to the most similar question
most_similar_answer = data[max_index]['answer']

print("Most relevant answer:", most_similar_answer)


Most relevant answer: When Mane will share confidential information with another person or entity.  Also, when someone will observe or learn about Mane's confidential information. Additionally, when any of the parties (MANE and Counterparty) wish to disclose confidential information to each other.


In [7]:

user_question = input("Ask a question: ")
threshold = 0.8
similarities = []
for i, item in enumerate(data):
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)

    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding).item()

    # Display only if similarity score is above the threshold
    if cosine_sim > threshold:
        print(f"Question {i + 1}: {item['question']}")
        print(f"Cosine Similarity Score: {cosine_sim}")
        print()

        # Store the similarity score
        similarities.append(cosine_sim)

# Check if there are any similarities above the threshold
if similarities:
    # Sort the similarities in descending order
    sorted_similarities = sorted(similarities, reverse=True)

    # Find the index of the question with the highest cosine similarity
    max_index = similarities.index(sorted_similarities[0])

    # Get the answer corresponding to the most similar question
    most_similar_answer = data[max_index]['answer']

    print("Most relevant answer:", most_similar_answer)
    print("Cosine Similarity Score with most relevant question:", sorted_similarities[0])

    # Check the difference between the top two similarity scores
    score_difference = sorted_similarities[0] - sorted_similarities[1]

    # Set a threshold for prompting the user for more specificity
    specificity_threshold = 0.05

    if score_difference < specificity_threshold:
        print("The similarity scores are very close. Please be more specific in your question.")
else:
    print("No relevant questions found above the threshold.")


Question 36: When is a CDA needed?
Cosine Similarity Score: 0.9119568467140198

Question 50: Is a CDA needed for a facility visit?
Cosine Similarity Score: 0.8341062664985657

Question 78: What BI means in a CDA?
Cosine Similarity Score: 0.8040212988853455

Most relevant answer: A MANE Commercial Representative is appointed to assist for the promotion and sale of MANE's products (fragrances/ flavours/ ingredients), within a limited territory (country, region, city) for a specific market (fragrance/ flavour for the food/ cosmetic/ personal care/ home care, and less frequently, in fine fragrance, pharmaceutical, tobacco, pet food, and oral care industries), and grow MANE's direct sales accordingly. A MANE Distributor is appointed to resell 'as is' some of MANE's products (fragrances/ flavours/ ingredients), within a limited territory (country, region, city) for a specific market (fragrance/ flavour for the food/ cosmetic/ personal care/ home care, and less frequently, in fine fragrance, 

In [2]:
pip install spacy

  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/90/f0/0133b684e18932c7bf4075d94819746cee2c0329f2569db526b0fa1df1df/spacy-3.7.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/c1/c3/dd044e6f62a3d317c461f6f0c153c6573ed13025752d779e514000c15dd2/cymem-2.0.8-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for preshed<3.1.0,>=3.0.2 from https://files.pythonhosted.org/packages/e4/fc/78cdbdb79f5

**Bert base model**

In [13]:
import json
import torch
import spacy
from transformers import BertTokenizer, BertModel
from difflib import SequenceMatcher
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

nlp = spacy.load("en_core_web_sm")

with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)
user_question = input("Ask a question: ")

user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

similarities = []
for i, item in enumerate(data):
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)

    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding).item()
    similarities.append(cosine_sim)

top_similar_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:2]

for idx in top_similar_indices:
    print(f"Question {idx + 1}: {data[idx]['question']}")
    print(f"Cosine Similarity Score: {similarities[idx]}")

user_response = input("Do you want to clarify your question for better results? (yes/no): ")

if user_response.lower() == 'yes':
    prev_question = data[top_similar_indices[0]]['question']
    refined_question = input("Please refine your question: ")

    refined_tokens = nlp(refined_question)
    refined_lemmas = " ".join([token.lemma_ for token in refined_tokens])

    prev_tokens = nlp(prev_question)
    prev_lemmas = " ".join([token.lemma_ for token in prev_tokens])

    lemmatized_question = f"{refined_lemmas} {prev_lemmas}"
    lemmatized_tokens = tokenizer.encode(lemmatized_question, truncation=True, max_length=512, return_tensors='pt')
    lemmatized_embedding = model(input_ids=lemmatized_tokens).last_hidden_state.mean(dim=1)

    prev_tokens = tokenizer.encode(prev_question, truncation=True, max_length=512, return_tensors='pt')
    prev_embedding = model(input_ids=prev_tokens).last_hidden_state.mean(dim=1)
    lemmatized_cosine_sim = torch.nn.functional.cosine_similarity(lemmatized_embedding, prev_embedding).item()

    print(f"Cosine Similarity Score with refined question: {lemmatized_cosine_sim}")
    if lemmatized_cosine_sim > 0.8:
        print("Most relevant answer:", data[top_similar_indices[0]]['answer'])
    else:
        print("No relevant answer found for the refined question.")


Question 20: When should I involve the Legal & IP Department?
Cosine Similarity Score: 0.8356700539588928
Question 22: What is the role of the Legal & IP Department in the contract review process?
Cosine Similarity Score: 0.8273530006408691
Cosine Similarity Score with refined question: 0.8484472036361694
Most relevant answer: The Legal & IP Department (the Legal Counsel previously involved in similar contracts with the same counterparty) must be copied on the correspondence regarding the contract review between all the involved departments, but the contract will only be submitted to the Legal & IP Department once all the required validations and comments are obtained.


In [17]:
pip install transformers==4.12.2

     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
      --------------------------------------- 0.0/3.1 MB 178.6 kB/s eta 0:00:17
      --------------------------------------- 0.1/3.1 MB 252.2 kB/s eta 0:00:12
     -- ------------------------------------- 0.2/3.1 MB 551.6 kB/s eta 0:00:06
     ----- ---------------------------------- 0.4/3.1 MB 1.1 MB/s eta 0:00:03
     ------ --------------------------------- 0.5/3.1 MB 1.4 MB/s eta 0:00:02
     --------- ------------------------------ 0.7/3.1 MB 1.7 MB/s eta 0:00:02
     ----------- ---------------------------- 0.9/3.1 MB 1.9 MB/s eta 0:00:02
     ------------- -------------------------- 1.0/3.1 MB 2.0 MB/s eta 0:00:02
     ----------

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers
      creating build\lib.win-amd64-cpython-311\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\models
      creating build\lib.win-amd64-cpython-311\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\decoders
      creating build\lib.win-amd64-cpython-311\tokenizers\normalizers
      copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\normalizers
      creating build\lib.

**Bert large model**

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained('bert-large-uncased')

nlp = spacy.load("en_core_web_sm")

with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)
user_question = input("Ask a question: ")

user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

similarities = []
for i, item in enumerate(data):
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)
    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding, dim=1).item()
    similarities.append(cosine_sim)
top_similar_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:2]
for idx in top_similar_indices:
    print(f"Question {idx + 1}: {data[idx]['question']}")
    print(f"Cosine Similarity Score: {similarities[idx]}")

user_response = input("Do you want to clarify your question for better results? (yes/no): ")

if user_response.lower() == 'yes':
    prev_question = data[top_similar_indices[0]]['question']
    refined_question = input("Please refine your question: ")
    refined_tokens = nlp(refined_question)
    refined_lemmas = " ".join([token.lemma_ for token in refined_tokens])

    prev_tokens = nlp(prev_question)
    prev_lemmas = " ".join([token.lemma_ for token in prev_tokens])

    lemmatized_question = f"{refined_lemmas} {prev_lemmas}"
    lemmatized_tokens = tokenizer.encode(lemmatized_question, truncation=True, max_length=512, return_tensors='pt')
    lemmatized_embedding = model(input_ids=lemmatized_tokens).last_hidden_state.mean(dim=1)

    prev_tokens = tokenizer.encode(prev_question, truncation=True, max_length=512, return_tensors='pt')
    prev_embedding = model(input_ids=prev_tokens).last_hidden_state.mean(dim=1)
    lemmatized_cosine_sim = torch.nn.functional.cosine_similarity(lemmatized_embedding, prev_embedding, dim=1).item()

    print(f"Cosine Similarity Score with refined question: {lemmatized_cosine_sim}")
    if lemmatized_cosine_sim > 0.8:
        print("Most relevant answer:", data[top_similar_indices[0]]['answer'])
    else:
        print("No relevant answer found for the refined question.")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

c:\Users\wissa\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wissa\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Question 2: What is the difference between a Commercial Representative and an Agent?
Cosine Similarity Score: 0.9401176571846008
Question 1: What is the difference between a Commercial Representative and a Distributor?
Cosine Similarity Score: 0.9188472628593445
Cosine Similarity Score with refined question: 0.9340527653694153
Most relevant answer: An Agent is entitled to negotiate selling prices on behalf of the company it represents, while a Commercial Representative is not. MANE does not appoint Agents, only Commercial Representatives.
